In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages
! pip install transformers datasets
! pip install sentencepiece
! pip install transformers[torch]
! pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.3 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [1]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

left_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_left_fulltext.csv')
left_dataset = left_dataset['train']

ModuleNotFoundError: ignored

In [ ]:
# split data into trainings and test split
left_dataset = left_dataset.train_test_split(test_size=0.2)

In [ ]:
left_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'url', 'date', 'title', 'tags', 'heading', 'source', 'preview_text', 'full_text', 'bias_rating', 'full_text_cleaned'],
        num_rows: 8540
    })
    test: Dataset({
        features: ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'url', 'date', 'title', 'tags', 'heading', 'source', 'preview_text', 'full_text', 'bias_rating', 'full_text_cleaned'],
        num_rows: 2136
    })
})

## Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text_cleaned"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_left_dataset = left_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=left_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1559 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5879 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1762 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1146 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1177 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1118 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1275 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1448 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 32

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_left_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# add pad token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

## Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import PegasusForCausalLM

model = PegasusForCausalLM.from_pretrained("google/pegasus-large", add_cross_attention=False)

Some weights of PegasusForCausalLM were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True <class 'transformers.models.pegasus.modeling_pegasus.PegasusForCausalLM'> has to be configured as a decoder.


In [ ]:
#finetune the pretrained model on the dataset

training_args = TrainingArguments(
    output_dir="pegasus_left_fulltext_cleaned_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=20
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/tobijen/pegasus_left_fulltext_cleaned_torch into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,6.819901
2,No log,6.066168
3,No log,5.463881
4,No log,5.011125
5,No log,4.562147
6,No log,4.155547
7,No log,3.768315
8,No log,3.411103
9,No log,3.120401
10,No log,2.922219


TrainOutput(global_step=660, training_loss=4.330995733087713, metrics={'train_runtime': 104.4565, 'train_samples_per_second': 50.547, 'train_steps_per_second': 6.318, 'total_flos': 272446783488000.0, 'train_loss': 4.330995733087713, 'epoch': 20.0})

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 8.20


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.37G [00:00<?, ?B/s]

Upload file runs/Jul28_15-25-55_157265a02823/events.out.tfevents.1690557965.157265a02823.7193.0:   0%|        …

Upload file runs/Jul28_15-25-55_157265a02823/events.out.tfevents.1690558069.157265a02823.7193.1:   0%|        …

To https://huggingface.co/tobijen/pegasus_left_fulltext_cleaned_torch
   dd65372..e854ebc  main -> main

   dd65372..e854ebc  main -> main

To https://huggingface.co/tobijen/pegasus_left_fulltext_cleaned_torch
   e854ebc..66cc888  main -> main

   e854ebc..66cc888  main -> main



'https://huggingface.co/tobijen/pegasus_left_fulltext_cleaned_torch/commit/e854ebc50b70d0f5ce65dcc02088cf218496d400'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/pegasus_left_fulltext_cleaned_torch")

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Come up with a prompt you'd like to generate text from:

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for text generation with your model, and pass your text to it:

In [ ]:
# generate text with the pipeline module from the transformers library
from transformers import pipeline

generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/pegasus_left_fulltext_cleaned_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=50)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)


social_1  =>  Gay marriage is


[{'generated_text': 'Gay marriage is gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay'}]
social_2  =>  Abortion is
[{'generated_text': 'Abortion is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is'}]
education_1  =>  Public education is
[{'generated_text': 'Public education is is is is is is is is is is is is is is is is is is is is is is is is is is public public public Public Public Public Public Public Public Public Public Public Public Public Public Public Public Public Public Public Public Public Public Public'}]
education_2  =>  Charter schools are
[{'generated_text': 'Charter schools are are are in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in'}]
e

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)


data_to_append = {
    "pegasus_left_fulltext_cleaned": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)